# 7. 텍스트 데이터 다루기
# Working With Text Data

## 7.1 문자열 데이터 타입

데이터 분석을 하면서 문자열 데이터를 접할 수 있는 경우는 아래와 같이 네 가지 종류가 있다.
- 범주형 데이터
- 범주에 의미를 연결시킬 수 있는 임의의 문자열
- 구조화된 문자열 데이터
- 텍스트 데이터

__범주형 데이터__는 고정된 목록(개수)로 구성된다. 예를 들어 가장 좋아하는 색을 묻는 설문에서 `빨강, 녹색, 파랑`과 같이 하나를 선택하는 경우를 말한다.<br />
__범주에 의미를 연결시킬 수 있는 임의의 문자열은__ "사과 같이 빨간색", "불그스름한 색" 은 "빨강"이라는 범주에 연결시킬 수 있는 문자열을 의미한다. <br />
__구조화된 문자열 데이터__는 범주형 데이터 같은 데이터는 아니지만, 주소나 장소, 이메일, 전화번호 등 처럼 일정한 구조를 가진 데이터를 말한다. <br />
__텍스트 데이터__는 뉴스의 기사, 댓글, 논문 등과 같이 일반적인 텍스트 데이터를 말한다. 

텍스트 분석에서 데이터 셋을 ***말뭉치(corpus)***라 하고, 하나의 텍스트를 의미하는 각 데이터 포인트를 ***문서(document)***라고 한다. 

## 7.2 예제 애플리케이션: 영화 리뷰 감성 분석 (Sentiment Analysis)
이 교재에서는 텍스트 분석 예제로 스탠포드 대학교 연구원인 앤드류 마스가 IMDB(Internet Movie DataBase)에서 수집한 영화 리뷰 데이터셋을 사용한다. 해당 데이터 셋은 http://ai.stanford.edu/~amaas/data/sentiment/ 에서 다운 받을 수 있다. 다운로드 후 압축을 풀면 약 400MB 크기의 데이터가 있고 아래와 같이 구성되어 있다. IMDb 사이트에는 1 ~ 10까지 점수가 있다. 이 데이터셋은 7점이상은 positive, 4점 이하는 negative인 이진 분류 데이터셋으로 구분 되어있다.

In [1]:
## Windows10 - 64bit
# !tree data/aclImdb

## Mac OS
!tree -dL 2 data/aclImdb

data/aclImdb
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    ├── pos
    └── unsup

7 directories


`pos`폴더에는 긍정적인(positive) 리뷰가 각각 하나의 파일로 나위어 있고, `neg` 폴더(negative)에도 마찬가지로 있다. `unsup` 폴더는 레이블이 없는 데이터를 담고 있는데, 실습에서는 필요하지 않으므로 삭제해준다. 

In [3]:
# Mac OS
!rm -r data/aclImdb/train/unsup

rm: data/aclImdb/train/unsup: No such file or directory


`scikit-learn`의 `load_files`함수를 사용해서 파일을 읽어 올 수 있다. `load_files`로 폴더의 데이터를 읽을 때 레이블은 폴더의 알파벳 순서에 따라 0부터 부여한다. 여기서의 데이터셋은 neg 폴더에는 레이블이 0이되고, pos는 1이 된다.

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from IPython.display import display

In [7]:
from sklearn.datasets import load_files

%time reviews_train = load_files('./data/aclImdb/train/')

CPU times: user 516 ms, sys: 383 ms, total: 900 ms
Wall time: 905 ms


In [10]:
text_train, y_train = reviews_train.data, reviews_train.target
print('text_train의 타입 : {}'.format(type(text_train)))
print('text_train의 길이 : {}'.format(len(text_train)))
print('text_train[6]: \n{}'.format(text_train[6]))

text_train의 타입 : <class 'list'>
text_train의 길이 : 25000
text_train[6]: 
b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."


In [11]:
# html 태그 제거 - <br />태그
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [12]:
print('클래스별 샘플 수 (Training data): {}'.format(np.bincount(y_train)))

클래스별 샘플 수 (Training data): [12500 12500]


이번에는 같은 방법으로 Test data를 가져온다.

In [14]:
%time reviews_test = load_files("./data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("테스트 데이터의 문서 수: {}".format(len(text_test)))
print("클래스별 샘플 수 (테스트 데이터): {}".format(np.bincount(y_test)))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

CPU times: user 503 ms, sys: 370 ms, total: 872 ms
Wall time: 877 ms
테스트 데이터의 문서 수: 25000
클래스별 샘플 수 (테스트 데이터): [12500 12500]


## 7.3 텍스트 데이터를 BOW로 표현하기
머신러닝에서 텍스트를 표현하는 방법 중 ***BOW(Bag Of Words)*** 는 간단하면서도 효과적이어서 많이 사용한다. 하지만, BOW를 사용할 경우, 텍스트 데이터는 텍스트 자체의 문장, 문단과 같은 텍스트 구조는 사라지게 되며 단어의 빈도수만 확인 가능하다. BOW를 표현하기 위해서는 아래의 세 가지 단계가 필요하다. <br />
1. **토큰화(tokenization)**: 각 문서(문장)을 단어(토큰)으로 나누는 작업을 말한다.
2. **어휘 사전 구축**: 모든 문서에 나타난 모든 단어의 어휘를 모으고 번호를 매긴다(알파벳 순서)
3. **인코딩**: 어휘 사전의 단어가 문서마다 몇 번이나 나타나는지를 계산한다.

### 7.3.1 샘플 데이터에 BOW 적용하기

In [15]:
# 영화 리뷰 데이터가 아닌 임의의 데이터

bards_words = ["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
print("어휘 사전의 크기: {}".format(len(vect.vocabulary_)))
print("어휘 사전의 내용:\n {}".format(vect.vocabulary_))

어휘 사전의 크기: 13
어휘 사전의 내용:
 {'be': 0, 'is': 6, 'to': 11, 'doth': 2, 'the': 9, 'himself': 5, 'but': 1, 'he': 4, 'knows': 7, 'man': 8, 'think': 10, 'fool': 3, 'wise': 12}


샘플데이터에 대해 BOW형태로 만들려면 `transform`메소드를 호출한다.

In [19]:
bag_of_words = vect.transform(bards_words)
print("BOW: {}".format(repr(bag_of_words)))

BOW: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [20]:
print("BOW의 밀집 표현:\n{}".format(bag_of_words.toarray()))

BOW의 밀집 표현:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


### 7.3.2 영화 리뷰에 대한 BOW

In [22]:
%%time
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))

X_train:
<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>
CPU times: user 12.2 s, sys: 221 ms, total: 12.4 s
Wall time: 12.5 s
